In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
import LoadDataset as LD
import numpy as np
import scipy.misc
from scipy.misc import imresize
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

Using TensorFlow backend.


ModuleNotFoundError: No module named 'skimage'

In [2]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 5
print(epochs)

5


In [4]:
n_categories = 3
categories = np.array(["c0", "c1"])

load_data = LD.LoadData('/netscratch/data/state_farm/imgs/train') # give the directory address.
X_dat ,Y_dat = load_data.generate_data()
X_dat, Y_dat = load_data.shuffle_data(X_dat,Y_dat)

Dataset directory added
['c6', 'c4', 'c5', 'c8', 'c1', 'c0', 'c7', 'c3', 'c9', 'c2']
c6  directory images are fetched!!! 
c4  directory images are fetched!!! 
c5  directory images are fetched!!! 
c8  directory images are fetched!!! 
c1  directory images are fetched!!! 
c0  directory images are fetched!!! 
c7  directory images are fetched!!! 
c3  directory images are fetched!!! 
c9  directory images are fetched!!! 
c2  directory images are fetched!!! 


In [ ]:
X = np.array(X_dat)
Y = np.array(Y_dat)
X.shape

(22424, 480, 640, 3)

In [ ]:
# this block does the image resizing
i_width = 227
i_height = 227
out = []

for i in range(0,22424):
    img = X[i, :, :, :]
#     print(img.shape)
    out.append(resize(img, (i_height, i_width, 3)))
out = np.array(out)
print(out.shape)


In [17]:
(x_train_img, y_train_img) = (out[0:22000, 0:227, 0:227,], Y[0:22000])
(x_test_img, y_test_img) = (X[22000:, 0:227, 0:227,], Y[22000:])

resize_image = []
begin = 0
for i in range(0,22):
    stop = begin + 1000
    img = x_train_img[begin:stop, :, :, :]
    begin = stop
    image_resized = resize(img, (img.shape[0], img.shape[1] / 4, img.shape[2] / 4, img.shape[3]), anti_aliasing=True)
    # image_downscaled = downscale_local_mean(X, (4, 3))
    image_resized.shape

KeyboardInterrupt: 

In [31]:
(x_train, y_train) = (out[0:2000,], Y[0:2000])
(x_test, y_test) = (out[20000:,], Y[20000:])
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train.shape)

227 227 3
(2000, 227, 227, 3)


In [27]:
# for cifar10 dataset
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)

img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [32]:
# AlexNet Define the Model
model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='valid', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
model.add(Conv2D(96, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(256, (5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [33]:
# print the model summary
model.summary()

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Train the Model

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 114, 114, 96)      2688      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 57, 57, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 57, 57, 96)        384       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 57, 57, 256)       614656    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 28, 28, 256)       0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 28, 28, 256)       1024      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 384)       885120    
__________

ResourceExhaustedError: OOM when allocating tensor of shape [43264,4096] and type float
	 [[node training_3/Adam/zeros_16 (defined at /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:702)  = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [43264,4096] values: [0 0 0...]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'training_3/Adam/zeros_16', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python

In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0186121646881103
Test accuracy: 0.6857
